# Analysis
This notebook creates features from raw tables and visualizes the results.

## 1. Set environment
Import libraries

In [1]:
import numpy as np
import pandas as pd
from math import floor
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from wordcloud import WordCloud, STOPWORDS
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import statsmodels.api as sm
from isodate import parse_duration
from scipy.stats import ttest_ind
from stargazer.stargazer import Stargazer

The following cell parses json files. Avoid running it again.

Counts:
- All videos
    - 1,928
- All videos with at least one comment in first 12 hours
    - 1,846
- All videos with at least one comment in English in first 12 hours
    - 1,814
- All videos with at least one comment in first 12 hours and excluding fuzzy window
    - 1533
- All videos with at least one comment in English in first 12 hours excluding fuzzy window
    - ?
- All comments
    - 1,197,454

Read datasets

In [2]:
d1 = pd.read_csv('../../dat/videoDetails.csv')
d2 = pd.read_csv('../../dat/videoFlagsFuzzy.csv')
d3 = pd.read_csv('../../dat/videoFlagsFuzzyLangid.csv')
df = pd.merge(d1, d3, on='videoId', how='right')

In [12]:
start = pd.Timestamp('2021-11-10 00:00:00')
start - pd.Timedelta(hours=12)

Timestamp('2021-11-09 12:00:00')

In [17]:
df.head(2)

,videoId,definition,description,duration,liveBroadcastContent,publishedAt,tags,title,post12CommentsNum,post12CommentsNeg1,...,post60CommentsNum,post60CommentsNeg1,post60CommentsNeg2,post60CommentsPos1,post60CommentsPos2,post72CommentsNum,post72CommentsNeg1,post72CommentsNeg2,post72CommentsPos1,post72CommentsPos2
0,ewKg8DkwcLc,hd,CNN obtained exclusive video that shows Trump ...,PT8M11S,none,2021-11-05T12:01:03Z,"latest News, Happening Now, CNN, Chris Cuomo, ...",Rudy Giuliani challenged under oath on his ele...,3074,1490,...,4130,2040,2615,1183,2085,4305,2129,2724,1232,2176
1,E4GXPtsHwKE,hd,"A year after the 2020 election, former Preside...",PT5M47S,none,2021-11-05T15:23:25Z,"latest News, Happening Now, CNN, John Avlon, R...",'A cut and paste job': Avlon lays out Trump's ...,955,514,...,1296,701,943,395,733,1383,751,1007,417,776


In [13]:
# Convert publishedAt to timestamp
df['publishedAt'] = pd.to_datetime(df['publishedAt'], format='%Y-%m-%dT%H:%M:%SZ')

# Start of policy rollout
start = pd.Timestamp('2021-11-10 00:00:00')

# Time windows
windows = np.arange(12, 72+1, 12)

# Init dict in which to store masks
m = {}
for h in windows:
    lim = start - pd.Timedelta(hours=12)
    d = {
        f'h{h}': (df['publishedAt'] > lim) | (df['publishedAt'] >= '2021-11-11')
    }

TypeError: '>' not supported between instances of 'str' and 'Timestamp'

In [9]:
d = {}


Data summary

In [ ]:
nvids = d2['videoId'].nunique()
ncoms = round(df['post12CommentsNum'].sum(), -5)

print(f'Scraped {int(ncoms)} from {nvids} Political videos.')

## 2. Feature creation
Add constant term

In [ ]:
df['const'] = 1

Convert `publishedAt` to datetime format

In [ ]:
df['publishedAt'] = pd.to_datetime(df['publishedAt'], format='%Y-%m-%dT%H:%M:%SZ')

Turn `definition` to dummy

In [ ]:
df['definition'] = df['definition'].replace({'sd':'0','hd':'1'}).astype(int)

Create targets from counters

In [ ]:
# Hours (12, 24, 36, ...)
hours = list(np.arange(12, 72+1, 12))

# NCR and sNCR
for h in hours:
    df[f'ncr1Post{h}'] = df[f'post{h}CommentsNeg1'] / df[f'post{h}CommentsNum'].replace(0, 0.1)
    df[f'ncr2Post{h}'] = df[f'post{h}CommentsNeg2'] / df[f'post{h}CommentsNum'].replace(0, 0.1)

Analyze most important words in video titles to create dummy variables

In [ ]:
# All titles to single text
text = ' '.join(df['title'].str.title().tolist())

# Remove annoying strings
for string in ["'s",".","-"]:
    text = text.replace(string, '')

# All words to uppercase
text = text.upper()
# Stopwords
stopwords = set(list(STOPWORDS) + ['SAY','SAYS','S'])

# Plot wordcloud
wordcloud = WordCloud(
    background_color='white',
    max_words=25,
    stopwords=stopwords,
    max_font_size=40, 
    scale=3,
    random_state=42
).generate(text)

# Show wordcloud
fig = plt.figure(figsize=(12, 12))
plt.axis('off')
plt.imshow(wordcloud)
plt.show()

Create dummy variables by topic

In [ ]:
# Title to lowercase
df['title'] = df['title'].str.lower()

# Dictionary of keywords
topics = {
    'president':'biden|trump',
    'climate':'cop26|cop 26|climate',
    'economy':'inflation|infrastructure|bill|economy',
    'covid':'covid|covid19|covid-19|virus',
    'violence':'kill|murder|assassins| die|dead|shoot|shot'
}

# Create Indicator variables
for topic in topics.keys():
    df[topic] = np.where(df['title'].str.contains(topics[topic]), 1, 0)

Video title sentiment

In [ ]:
clf = SentimentIntensityAnalyzer()
df['toneCom'] = df['title'].apply(lambda x: clf.polarity_scores(x)['compound'])
df['tonePos'] = df['title'].apply(lambda x: clf.polarity_scores(x)['pos'])
df['toneNeg'] = df['title'].apply(lambda x: clf.polarity_scores(x)['neg'])

Translate `duration` to seconds.

In [ ]:
# YT-duration format to seconds
df['seconds'] = df['duration'].apply(lambda x: int(parse_duration(x).total_seconds()))

# log(seconds)
df['logSeconds'] = np.log(df['seconds'])

Sort data by upload date

In [ ]:
df = df.sort_values('publishedAt', ascending=True).reset_index(drop=True)

Treatment indicator

In [ ]:
df['treat'] = (df['publishedAt'] > '2021-11-10').astype(int)

Declare running variable $R_i$ and interaction term $R_i \times T_i$
- Before: Seconds until treatment (control was positive, treatment was negative)
- Update: Seconds since treatment (control is negative, treatment is positive)

In [ ]:
# Running variable
df['r'] = (df['publishedAt'] - pd.Timestamp('2021-11-10')).dt.total_seconds()

# Interaction
df['rTreat'] = df['r'].multiply(df['treat'])

## 3. Balance tests
List of targets

In [ ]:
windows = [f'Post{h}' for h in hours]

### 3.1. Descriptive statistics
Number of available videos as a function of $h$

In [ ]:
# post{h}CommentsNum
cols = [f'post{str(h)}CommentsNum' for h in hours]

# Merge to get videoId & post{h}CommentsNum
t = pd.merge(d1[['videoId','publishedAt']], d2[['videoId'] + cols], on='videoId', how='left')

# Create C&T groups
t['treat'] = (t['publishedAt'] > '2021-11-10').astype(int)

# Count available videos per window
t[cols] = t[cols].notna().astype(int)

# Group by
t = t.groupby('treat').agg({**{'videoId':'size'}, **dict(zip(cols,['sum']*6))}).transpose()

# Format
t.index = ['Total videos'] + ['h = ' + str(i*12) for i in range(1,7)]
t

# To latex
# print(t.to_latex(caption='Number of available videos for different values of $h$',
#                  label='tab_dat_nobs'))

### 3.2. Balance

Balance table for videos closest to the cutoff: $R_{hours} \in [-36,-12] \cup [24,36]$

1. Difference in means for binary covariates
1. RDD for continuous covariates

$$X_i = \gamma_0 + \gamma_1 r_i + \gamma_2 T_i + \gamma_3 r_i T_i + V_i$$

In [ ]:
# Bandwidth
mask = df['r'].between(-36*60*60, -12*60*60) | df['r'].between(24*60*60, 48*60*60)
print(f'{mask.sum()} videos used in balance test.')
d = df[mask].copy()

d['durationMins'] = d['seconds'].div(60)

# Order frequent-word variables by frequency
X = list(topics.keys()) + ['definition','durationMins','tonePos','toneNeg','toneCom']

# Regress each variable on r and treat
data = []
for x in X:
    m = sm.OLS(endog=d[x], exog=d[['const','r','treat','rTreat']]).fit()
    data.append((m.params['treat'], m.pvalues['treat']))

# Summary table
t = pd.DataFrame(data=data, index=X, columns=['Estimated Value','p-value'])
t.index.rename('Covariate', inplace=True)
t.reset_index(inplace=True)
t.round(3)

# To latex
# print(t.to_latex(caption='Regression discontinuities on observable characteristics',
#                  label='tab_dat_balance', float_format='%.3f', index=False))

## 4. Regression Analysis
Fit all polynomial models with $d \in \{1, 2\}$.

### 4.1. First-degree
$Y_i = \beta_0 + \beta_1 r_i + \beta_2 T_i + \beta_3 T_i \times r_i + \gamma X_i + U_i$

In [ ]:
# Minumum number of comments
mask = df['post12CommentsNum'] > 0

# Fit all models
d1 = []
for target in ['ncr1','ncr2']:
    for window in windows:
        formula = f'{target}{window} ~ treat + r + I(r * treat)'
        m = sm.OLS.from_formula(formula=formula, data=df[mask]).fit(cov_type='HC0')
        d1.append(m)

$NCR(h)$ summary

In [ ]:
# Summaries
ncr1d1 = Stargazer(d1[:6])
ncr1d1.dependent_variable_name('Dependent Variable: Negative Comment Ratio')
ncr1d1.custom_columns([f'h = {h}' for h in hours], [1]*6)
ncr1d1.covariate_order(['treat','r','I(r * treat)','Intercept'])#,'toneNeg'])
ncr1d1.rename_covariates({'I(r * treat)':'RxT','r':'R','treat':'T'})#,'toneNeg':'Negative tone'})
ncr1d1.show_degrees_of_freedom(False)
ncr1d1.add_custom_notes(['Robust standard errors (HC0)'])
ncr1d1.show_model_numbers(False)
ncr1d1.title('Estimated effects on Negative Comment Ratio (first-degree polynomial)')
ncr1d1

# Latex output
# print(ncr1d1.render_latex())

$sNCR(h)$ summaries

In [ ]:
# Model summaries
ncr2d2 = Stargazer(d1[6:])
ncr2d2.dependent_variable_name('Dependent Variable: Somewhat Negative Comment Ratio')
ncr2d2.custom_columns([f'h = {h}' for h in hours], [1]*6)
ncr2d2.covariate_order(['treat','r','I(r * treat)','Intercept'])#,'toneNeg'])
ncr2d2.rename_covariates({'I(r * treat)':'RxT','r':'R','treat':'T'})#,'toneNeg':'Negative tone'})
ncr2d2.show_degrees_of_freedom(False)
ncr2d2.add_custom_notes(['Robust standard errors (HC0)'])
ncr2d2.show_model_numbers(False)
ncr2d2.title('Estimated effects on Somewhat Negative Comment Ratio (first-degree polynomial)')
ncr2d2

# Latex output
# print(ncr2d2.render_latex())

### 4.2. Second-degree
$$Y_i = \beta_0 + \beta_1 T_i + \beta_2 R_i + \beta_3 R_i^2 + \beta_4 (T_i \times R_i) + \beta_5 (T_i \times R_i^2) + \gamma X_i + U_i$$

In [ ]:
# Minumum number of comments
mask = df['post12CommentsNum'] > 0

# Fit all models
d2 = []
for target in ['ncr1','ncr2']:
    for window in windows:
        formula = f'{target}{window} ~ treat + r + I(r**2) + I(treat * r) + I(treat * (r**2))'
        m = sm.OLS.from_formula(formula=formula, data=df[mask]).fit(cov_type='HC0')
        d2.append(m)

$NCR(h)$ summary

In [ ]:
# Summaries
ncr1d2 = Stargazer(d2[:6])
ncr1d2.dependent_variable_name('Dependent Variable: Negative Comment Ratio')
ncr1d2.custom_columns([f'h = {h}' for h in hours], [1]*6)
ncr1d2.covariate_order(['treat','r','I(r ** 2)','I(treat * r)','I(treat * (r ** 2))','Intercept'])
ncr1d2.rename_covariates({
    'I(r ** 2)':'R^2', 'I(treat * (r ** 2))':'T x R^2', 'I(treat * r)':'T x R',
    'r':'R','treat':'T'})
ncr1d2.show_degrees_of_freedom(False)
ncr1d2.add_custom_notes(['Robust standard errors (HC0)'])
ncr1d2.show_model_numbers(False)
ncr1d2.title('Estimated effects on Negative Comment Ratio (second-degree polynomial)')
ncr1d2

# Latex output
# print(ncr1d2.render_latex())

$sNCR(h)$ summaries

In [ ]:
# Summaries
ncr1d2 = Stargazer(d2[:6])
ncr1d2.dependent_variable_name('Dependent Variable: Somewhat Negative Comment Ratio')
ncr1d2.custom_columns([f'h = {h}' for h in hours], [1]*6)
ncr1d2.covariate_order(['treat','r','I(r ** 2)','I(treat * r)','I(treat * (r ** 2))','Intercept'])
ncr1d2.rename_covariates({
    'I(r ** 2)':'R^2', 'I(treat * (r ** 2))':'T x R^2', 'I(treat * r)':'T x R',
    'r':'R','treat':'T'})
ncr1d2.show_degrees_of_freedom(False)
ncr1d2.add_custom_notes(['Robust standard errors (HC0)'])
ncr1d2.show_model_numbers(False)
ncr1d2.title('Estimated effects on Somewhat Negative Comment Ratio (second-degree polynomial)')
ncr1d2

# Latex output
# print(ncr1d2.render_latex())

### 4.3. Robustness checks
#### 4.3.1. Control for president

### 4.4. Visualizations
Linear and quadratic RDD for $NCR(12)$

In [ ]:
# Linrear and quadratic models
l, q = d1[0], d2[0]

# Data and predictions for lines
t = pd.DataFrame({'r':df.loc[mask, 'r'],
                  'l':d1[0].fittedvalues.values,
                  'q':d2[0].fittedvalues.values})
t['treat'] = (t['r'] > 0).astype(int)

# Data for scatter
bin_length = 4
s = df.loc[mask, ['ncr1Post12','r']].copy()
s['bin'] = (s['r'].div(60 * 60) / bin_length).apply(lambda x: floor(x))
s = s.groupby('bin')['ncr1Post12'].mean().reset_index(name='mean')
s['bin'] = s['bin'].multiply(bin_length * 60 * 60)

# Initialize figure
fig, axs = plt.subplots(nrows=1, ncols=2)
fig.set_figheight(5)
fig.set_figwidth(14)

# Plot model on each axis
for i, ax in enumerate(axs):
    # Plot linear model
    if i == 0:
        series = 'l'
    else:
        series = 'q'
    # Lines
    ax.plot(t.loc[t['treat'].eq(0), 'r'], t.loc[t['treat'].eq(0), series], color='C1')
    ax.plot(t.loc[t['treat'].eq(1), 'r'], t.loc[t['treat'].eq(1), series], color='C1')
    # Scatter
    ax.scatter(s['bin'], s['mean'], color='C0', alpha=0.5)
    # Shaded regions
    ax.axvspan(xmin=0, xmax=24*60*60, color='gray', alpha=0.3)
    ax.axvspan(xmin=-12*60*60, xmax=0, color='C0', alpha=0.3)
    # X&Y axes
    ax.set_xticks(np.arange(-5*24*60*60, 6*24*60*60+1, 24 * 60 * 60))
    ax.set_xticklabels(np.arange(-5*24, 6*24+1, 24), rotation=45)
    ax.set_xlim(-5*24*60*60, 6*24*60*60)
    ax.set_ylim(0, 0.7)
    # Labels
    title = {'l':'First-degree polynomial','q':'Second-degree polynomial'}
    ax.set_title(f'{title[series]}')
    ax.set_xlabel('Hours until policy')
    ax.set_ylabel('Negative Comment Ration (h=12)')
    ax.grid(which='major', axis='x')
# Save and show
if 'google.colab' not in sys.modules:
    plt.savefig('../../fig/fig_d1d2.png', dpi=200, bbox_inches='tight')
plt.show()

All linear models

In [ ]:
# Initialize figure
fig, axs = plt.subplots(nrows=6, ncols=2)
fig.set_figheight(20)
fig.set_figwidth(14)

# Plot within each axis
for i, ax_row in enumerate(axs):
    # Left-right plots
    for j, ax in enumerate(ax_row):
        if j == 0:
            outcome = f'ncr1{windows[i]}'
            dotColor = 'blue'
            ax.set_ylabel(f'NCR(h = {hours[i]})')
            model = d1[i]
        else:
            outcome = f'ncr2{windows[i]}'
            dotColor = 'lightblue'
            ax.set_ylabel(f'sNCR(h = {hours[i]})')
            model = d1[6+i]
        # Masks
        mask = df['post12CommentsNum'].gt(0) & df[outcome].notna()
        # Line plots
        t = df.loc[mask, ['treat','r']].assign(pred = model.fittedvalues)
        ax.plot(t.loc[t['treat'].eq(0), 'r'], t.loc[t['treat'].eq(0), 'pred'], color='C1')
        ax.plot(t.loc[t['treat'].eq(1), 'r'], t.loc[t['treat'].eq(1), 'pred'], color='C1')
        # Scatter plots
        s = df[mask].copy()
        s['bin'] = (s['r'].div(4*60*60)).apply(lambda x: floor(x))
        s = s.groupby('bin')[outcome].mean().reset_index(name='mean')
        s['bin'] = s['bin'] * (4*60*60)
        ax.scatter(x=s['bin'], y=s['mean'], color='C0', alpha=0.5)
        # Shades
        ax.axvspan(xmin=0, xmax=24*60*60, color='gray', alpha=0.3)
        ax.axvspan(xmin=-hours[i]*60*60, xmax=0, color='C0', alpha=0.3)
        # Axes
        ax.set_ylim(0, 0.7)
        ax.set_xticks(np.arange(-5*24*60*60, 6*24*60*60+1, 24*60*60))
        ax.set_xticklabels(np.arange(-5*24, 6*24+1, 24))
        ax.grid(which='major', axis='x')
        if i == 0:
            ax.set_title({0:'Negative Comment Ratio',1:'Somewhat Negative Comment Ratio'}[j])
        if i == 5:
            ax.set_xlabel('Hours until policy')
# Show & save
if 'google.colab' not in sys.modules:
    plt.savefig('../../fig/fig_res_d1all.png', dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
# Initialize figure
fig, axs = plt.subplots(nrows=6, ncols=2)
fig.set_figheight(20)
fig.set_figwidth(14)

# Plot within each axis
for i, ax_row in enumerate(axs):
    # Left-right plots
    for j, ax in enumerate(ax_row):
        if j == 0:
            outcome = f'ncr1{windows[i]}'
            dotColor = 'blue'
            ax.set_ylabel(f'NCR(h = {hours[i]})')
            model = d2[i]
        else:
            outcome = f'ncr2{windows[i]}'
            dotColor = 'lightblue'
            ax.set_ylabel(f'sNCR(h = {hours[i]})')
            model = d2[6+i]
        # Masks
        mask = df['post12CommentsNum'].gt(0) & df[outcome].notna()
        # Line plots
        t = df.loc[mask, ['treat','r']].assign(pred = model.fittedvalues)
        ax.plot(t.loc[t['treat'].eq(0), 'r'], t.loc[t['treat'].eq(0), 'pred'], color='C1')
        ax.plot(t.loc[t['treat'].eq(1), 'r'], t.loc[t['treat'].eq(1), 'pred'], color='C1')
        # Scatter plots
        s = df[mask].copy()
        s['bin'] = (s['r'].div(4*60*60)).apply(lambda x: floor(x))
        s = s.groupby('bin')[outcome].mean().reset_index(name='mean')
        s['bin'] = s['bin'] * (4*60*60)
        ax.scatter(x=s['bin'], y=s['mean'], color='C0', alpha=0.5)
        # Shades
        ax.axvspan(xmin=0, xmax=24*60*60, color='gray', alpha=0.3)
        ax.axvspan(xmin=-hours[i]*60*60, xmax=0, color='C0', alpha=0.3)
        # Axes
        ax.set_ylim(0, 0.7)
        ax.set_xticks(np.arange(-5*24*60*60, 6*24*60*60+1, 24*60*60))
        ax.set_xticklabels(np.arange(-5*24, 6*24+1, 24))
        ax.grid(which='major', axis='x')
        if i == 0:
            ax.set_title({0:'Negative Comment Ratio',1:'Somewhat Negative Comment Ratio'}[j])
        if i == 5:
            ax.set_xlabel('Hours until policy')
# Show & save
if 'google.colab' not in sys.modules:
    plt.savefig('../../fig/fig_res_d2all.png', dpi=200, bbox_inches='tight')
plt.show()

Comparing linear to quadratic models

In [ ]:
# Goodness of fit
t = pd.DataFrame(
    {
        'd1R2a':[m.rsquared_adj for m in d1], 'd2R2a':[m.rsquared_adj for m in d2],
        'd1bic':[m.bic for m in d1], 'd2bic':[m.bic for m in d2],
        'd1aic':[m.aic for m in d1], 'd2aic':[m.aic for m in d2],
    }
)

t.round(4)